## Kafka_Setup Detail 

In [ ]:
---
- name: Setup Kafka and Zookeeper
  hosts: ubuntu_containers
  become: yes
  tasks:


- name: Playbook’a genel bir ad verir. Bu playbook, Kafka ve Zookeeper’ı kurup yapılandırmak için yazılmıştır.
- hosts: İşlem yapılacak sunucu grubunu belirtir. ubuntu_containers adlı gruptaki sunucular üzerinde işlem yapılacaktır.
- become: yes değeriyle çalıştırılarak root yetkisi (sudo) kullanılır; çünkü Kafka ve Zookeeper kurulumu için yönetici yetkisi gereklidir.
- tasks: Playbook’un içinde yapılacak işlemleri listeleyen anahtar kelimedir.

In [ ]:
Java Kurulumu
- name: Install Java
  apt:
    name: openjdk-11-jdk
    state: present


- apt: Ubuntu (veya Debian) tabanlı sistemlerde paketleri yönetmek için kullanılan modüldür.
- name: Yüklenecek paketin adı, burada openjdk-11-jdk yani Java 11 paketidir.
- state: Paketin durumu. present değeri, paketin mevcut olup olmadığını kontrol eder, yüklü değilse yükler.

In [ ]:
Kafka İndirme
yaml
Copy code
- name: Download Kafka
  get_url:
    url: https://archive.apache.org/dist/kafka/3.2.1/kafka_2.13-3.2.1.tgz
    dest: /tmp/kafka.tgz

- get_url: Belirtilen URL’den dosya indirmek için kullanılır.
- url: İndirilecek dosyanın adresi, burada Kafka’nın Apache arşivindeki konumu.
- dest: Dosyanın indirileceği konum, /tmp/kafka.tgz.

In [ ]:
Kafka Dosyalarını Çıkartma
yaml
Copy code
- name: Extract Kafka
  unarchive:
    src: /tmp/kafka.tgz
    dest: /opt/
    remote_src: yes

- unarchive: Sıkıştırılmış bir dosyayı çıkartmak için kullanılır.
- src: Çıkartılacak dosyanın konumu, /tmp/kafka.tgz.
- dest: Çıkartılacak hedef dizin, /opt/.
- remote_src: yes değeri, dosyanın sunucuda halihazırda bulunduğunu belirtir, yani dosya uzaktan kopyalanmaz.

In [ ]:
Kafka İçin Sembolik Bağlantı Oluşturma
yaml
Copy code
- name: Create symlink for Kafka
  file:
    src: /opt/kafka_2.13-3.2.1
    dest: /opt/kafka
    state: link

- file: Dosya ve dizin yönetimi için kullanılır, burada bir sembolik bağlantı oluşturulur.
- src: Sembolik bağlantının hedefi, kafka_2.13-3.2.1 klasörü.
- dest: Sembolik bağlantının konumu ve adı, /opt/kafka.
- state: link değeri, sembolik bağlantı oluşturulacağını belirtir.

In [ ]:
 Kafka Kullanıcısı ve Grubu Oluşturma
yaml
Copy code
- name: Create kafka user and group
  ansible.builtin.user:
    name: kafka
    comment: Kafka User
    create_home: no
    shell: /bin/false

- user: Kullanıcı yönetimi için kullanılır.
- name: Kullanıcı adı, burada kafka.
- comment: Kullanıcı hakkında bir açıklama, "Kafka User".
- create_home: no değeri, ev dizini oluşturulmayacağını belirtir.
- shell: Kullanıcının kabuk erişimi yok, bin/false olarak ayarlandı.

In [ ]:
Zookeeper Yapılandırma Dosyasını Kopyalama
yaml
Copy code
- name: Configure Zookeeper
  copy:
    dest: /opt/kafka/config/zookeeper.properties
    content: |
      dataDir=/var/lib/zookeeper
      clientPort=2181
      maxClientCnxns=0
      admin.enableServer=false

- copy: Dosya içeriğini kopyalamak veya oluşturmak için kullanılır.
- dest: Kopyalanacak veya oluşturulacak dosyanın konumu, burada zookeeper.properties.
- content: Dosyanın içeriği olarak bir dizi yapılandırma ayarı verilir.

In [ ]:
Kafka Yapılandırma Dosyasını Kopyalama
yaml
Copy code
- name: Configure Kafka
  copy:
    dest: /opt/kafka/config/server.properties
    content: |
      broker.id=0
      listeners=PLAINTEXT://:9092
      log.dirs=/var/lib/kafka
      zookeeper.connect=localhost:2181
      num.partitions=1
      offsets.topic.replication.factor=1
      transaction.state.log.replication.factor=1
      transaction.state.log.min.isr=1
      log.retention.hours=168
      log.segment.bytes=1073741824
      log.retention.check.interval.ms=300000

- dest: Kafka yapılandırmasının saklanacağı konum.
- content: Kafka’nın çalışması için gerekli yapılandırmaları içerir, broker ID, port, log ayarları, partition sayısı gibi ayarları tanımlar.

In [ ]:
Kafka ve Zookeeper Veri Dizinlerini Oluşturma
yaml
Copy code
- name: Create Kafka data directories
  file:
    path: "{{ item }}"
    state: directory
    owner: kafka
    group: kafka
  with_items:
    - /var/lib/zookeeper
    - /var/lib/kafka

- file: Dosya ve dizin oluşturmak için kullanılır.
- path: Dosya veya dizin yolu, {{ item }} ile with_items listesinde dönecek her öğe için çalışır.
- state: directory, belirtilen yolun bir dizin olacağını belirtir.
- owner ve group: Dizin sahibi ve grubu kafka kullanıcısı olarak atanır.

In [ ]:
 Zookeeper’ı Başlatma
yaml
Copy code
- name: Start Zookeeper
  command: /opt/kafka/bin/zookeeper-server-start.sh -daemon /opt/kafka/config/zookeeper.properties

- command: Belirtilen komutu çalıştırır.
- /opt/kafka/bin/zookeeper-server-start.sh -daemon /opt/kafka/config/zookeeper.properties: Zookeeper başlatma komutu. -daemon seçeneği, Zookeeper'ın arka planda çalışmasını sağlar.

In [ ]:
- name: Start Kafka
  command: /opt/kafka/bin/kafka-server-start.sh -daemon /opt/kafka/config/server.properties


-command: Kafka sunucusunu başlatmak için komut çalıştırır.
- /opt/kafka/bin/kafka-server-start.sh -daemon /opt/kafka/config/server.properties: Kafka sunucusunu başlatır, -daemon ile arka planda çalışmasını sağlar.

## Dockerfile Detail 


Dockerfile, Ubuntu tabanlı bir Docker imajı oluşturarak içine Python ve SSH sunucusu kurmayı amaçlar.

In [ ]:
FROM ubuntu:latest


FROM ubuntu
: Docker imajının temeli olarak en güncel Ubuntu sürümünü (ubuntu:latest) kullanır. Bu, Docker imajını sıfırdan oluşturmak yerine Ubuntu'nun en son sürümünden başlatacağımız anlamına gelir.

In [ ]:
Gerekli Paketleri Kurma
dockerfile
Copy code
RUN apt-get update && apt-get install -y \
    python3 \
    python3-pip \
    openssh-server \
    && apt-get clean

- RUN apt-get update: Paket listelerini günceller, böylece kurulum işlemi sırasında en güncel paketler kullanılır.
- && apt-get install -y: -y seçeneğiyle python3, python3-pip ve openssh-server paketlerini yükler:
- python3: Python 3 programlama dili için gereklidir.
- python3-pip: Python için paket yöneticisi, gerektiğinde Python modüllerini kurabilmek için kullanılır.
- openssh-server: SSH bağlantılarını kabul edebilmek için SSH sunucusunu kurar.
- && apt-get clean: Gereksiz dosyaları temizleyerek imaj boyutunu azaltır. Özellikle önbellek dosyaları temizlenir.

In [ ]:
SH Sunucusu İçin Gerekli Dizini Oluşturma


Copy code
RUN mkdir /var/run/sshd

- RUN mkdir /var/run/sshd: SSH sunucusunun çalışması için gerekli olan /var/run/sshd dizinini oluşturur.

In [ ]:
SSH İçin Root Kullanıcı Parolasını Ayarlama


Copy code
RUN echo 'root:password' | chpasswd

- RUN echo 'root
' | chpasswd: Root kullanıcısının şifresini password olarak ayarlar. chpasswd komutu, echo ile belirtilen kullanıcı adı ve parolayı alır ve şifreyi ayarlar.

In [ ]:
RUN sed -i 's/#PermitRootLogin prohibit-password/PermitRootLogin yes/' /etc/ssh/sshd_config


- RUN sed -i 's/#PermitRootLogin prohibit-password/PermitRootLogin yes/' /etc/ssh/sshd_config: PermitRootLogin ayarını yes olarak değiştirir. Bu ayar:
- PermitRootLogin yes: Root kullanıcısının SSH üzerinden giriş yapabilmesini sağlar. Varsayılan olarak bu ayar güvenlik için kapalıdır, ancak bu Dockerfile’da açılmıştır.

In [ ]:
SSH Sunucusunu Başlatma
dockerfile
Copy code
CMD ["/usr/sbin/sshd", "-D"]

- CMD ["/usr/sbin/sshd", "-D"]: SSH sunucusunu başlatır ve -D seçeneği ile arka planda çalıştırır. CMD komutu, konteyner başlatıldığında çalıştırılacak varsayılan komut olur.

In [ ]:
SSH İçin Portu Açma
dockerfile
Copy code
EXPOSE 22

- EXPOSE 22: Konteynerın dış dünya ile 22 numaralı SSH portu üzerinden iletişim kurmasına izin verir. EXPOSE, Docker’a bu port üzerinden iletişim sağlanabileceğini bildirir ancak otomatik olarak dışa açmaz; docker run -p komutu ile dışa açılabilir.